# Language modeling with Transformer
*Language Modeling refers to assigning a probability distribution for a given word (or a sequence of words) to follow a sequence of words.

![alt text](transformer_architecture.jpg "Title")

# Imports

In [1]:
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.utils.data import dataset
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from utils import *
import math, copy, time
from typing import Tuple
torch.manual_seed(547)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


# Data Pre-Process

In [2]:
train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>']) # represents rare tokens as <unk>

# train_iter was "consumed" by the process of building the vocab,
# so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
train_data, val_data, test_data = data_process(train_iter, vocab, tokenizer), data_process(val_iter, vocab, tokenizer), data_process(test_iter, vocab, tokenizer)

batch_size, eval_batch_size = 20, 10
# shape [seq_len, batch_size]
train_data, val_data, test_data = batchify(train_data, batch_size, device), batchify(val_data, eval_batch_size, device), batchify(test_data, eval_batch_size, device)

bptt = 35

# Model

In [3]:
class TransformerModel(nn.Module):

    def __init__(self, ntoken, d_model, nhead, d_hid, nlayers, dropout=0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.d_model = d_model
        self.encoder = nn.Embedding(ntoken, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, nlayers)
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self):
        # Pytorch default use Xavier
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output

In [4]:
class PositionalEncoding(nn.Module):
    """
    For every odd time-step create a vector using cos,
    for every even time-step create a vector using sin.
    Then add them to their corresponding embed vector. 
    """

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

# Configurations

In [5]:
ntokens = len(vocab)  # size of vocabulary
emsize  = 200  # embedding dimension
d_hid   = 200  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2    # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead   = 2    # number of heads in nn.MultiheadAttention
dropout = 0.2  # dropout probability
model   = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

# Loss and Optimizer

In [6]:
criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

# Training

In [7]:
def train(model):
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        batch_size = data.size(0)
        if batch_size != bptt:  # only on last batch
            src_mask = src_mask[:batch_size, :batch_size] # attend the earlier positions in the sequence
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5) # Grad norm
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | 'f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | 'f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

# Evaluation

In [8]:
def evaluate(model, eval_data):
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            batch_size = data.size(0)
            if batch_size != bptt:
                src_mask = src_mask[:batch_size, :batch_size]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += batch_size * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

# Main

In [ ]:
best_val_loss = float('inf')
epochs = 3
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss = evaluate(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | 'f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)

    scheduler.step()

| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 476.13 | loss  8.13 | ppl  3400.95
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch 495.39 | loss  6.88 | ppl   970.95
| epoch   1 |   600/ 2928 batches | lr 5.00 | ms/batch 475.89 | loss  6.43 | ppl   623.16
| epoch   1 |   800/ 2928 batches | lr 5.00 | ms/batch 483.62 | loss  6.29 | ppl   541.31
| epoch   1 |  1000/ 2928 batches | lr 5.00 | ms/batch 477.65 | loss  6.18 | ppl   485.40
| epoch   1 |  1200/ 2928 batches | lr 5.00 | ms/batch 489.08 | loss  6.16 | ppl   472.35
| epoch   1 |  1400/ 2928 batches | lr 5.00 | ms/batch 494.22 | loss  6.12 | ppl   452.84
| epoch   1 |  1600/ 2928 batches | lr 5.00 | ms/batch 487.94 | loss  6.11 | ppl   449.73
| epoch   1 |  1800/ 2928 batches | lr 5.00 | ms/batch 489.50 | loss  6.03 | ppl   414.40
| epoch   1 |  2000/ 2928 batches | lr 5.00 | ms/batch 479.59 | loss  6.02 | ppl   411.17
| epoch   1 |  2200/ 2928 batches | lr 5.00 | ms/batch 480.14 | loss  5.90 | ppl   364.93
| epoch   

# Evalutation on Test set for the Best Model

In [10]:
test_loss = evaluate(best_model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'The End: test loss {test_loss:5.2f}, 'f'test ppl: {test_ppl:8.2f}')
print('=' * 89)

The End: test loss  5.52, test ppl:   250.35
